## 3.4 データ集計 演習問題解答例

### 共通処理

In [1]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [2]:
# ライブラリのimport

# NumPy用ライブラリ
import numpy as np

# Matplotlib中のpyplotライブラリのインポート
import matplotlib.pyplot as plt

# matplotlib日本語化対応ライブラリのインポート
import japanize_matplotlib

# pandas用ライブラリ
import pandas as pd

# データフレーム表示用関数
from IPython.display import display

In [3]:
# 表示オプション調整

# NumPy表示形式の設定
np.set_printoptions(
    suppress=True, precision=4, floatmode='fixed'
)

# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14

# サイズ設定
plt.rcParams['figure.figsize'] = (6, 6)

# 方眼表示ON
plt.rcParams['axes.grid'] = True


# データフレームでの表示精度
pd.options.display.float_format = '{:.4f}'.format

# データフレームですべての項目を表示
pd.set_option("display.max_columns", None)

### ファイルダウンロード

In [4]:
 # URL指定
zip_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip'
fn = 'bank-full.csv'

# 公開データのダウンロードと解凍
!wget $zip_url
!unzip -o bank.zip

--2022-09-04 07:26:48--  https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 579043 (565K) [application/x-httpd-php]
Saving to: ‘bank.zip’

bank.zip            100%[===================>] 565.47K  1.63MB/s    in 0.3s    

2022-09-04 07:26:48 (1.63 MB/s) - ‘bank.zip’ saved [579043/579043]

Archive:  bank.zip
  inflating: bank-full.csv           
  inflating: bank-names.txt          
  inflating: bank.csv                


### CSV ファイル読み込み

In [5]:
# bank-full.csvをデータフレームに取り込み
df = pd.read_csv(
    fn,
    sep=';',
    na_values='unknown',
    true_values=['yes'],
    false_values=['no'])

### データ確認・加工

In [6]:
# 項目名の日本語定義
columns = [
    '年齢', '職業', '婚姻', '学歴', '債務不履行', '平均残高',
    '住宅ローン', '個人ローン', '連絡手段', '最終通話日',
    '最終通話月', '最終通話秒数', '通話回数_販促中',
    '前回販促後_経過日数', '通話回数_販促前', '前回販促結果',
    '今回販促結果'
]
# 項目名差し替え
df2 = df.copy()
df2.columns = columns

In [7]:
# 項目順番入れ替え
# 一番うしろの「今回販促結果」を一番前にする
# 「個人ローン」より後ろの項目を落とす
columns1 = list(df2.columns)
print(columns1)
columns2 = columns1[-1:] + columns1[:-9]
print(columns2)
df2 = df2[columns2]

['年齢', '職業', '婚姻', '学歴', '債務不履行', '平均残高', '住宅ローン', '個人ローン', '連絡手段', '最終通話日', '最終通話月', '最終通話秒数', '通話回数_販促中', '前回販促後_経過日数', '通話回数_販促前', '前回販促結果', '今回販促結果']
['今回販促結果', '年齢', '職業', '婚姻', '学歴', '債務不履行', '平均残高', '住宅ローン', '個人ローン']


### データ内容確認

In [8]:
# データ内容確認
display(df2.head())

,今回販促結果,年齢,職業,婚姻,学歴,債務不履行,平均残高,住宅ローン,個人ローン
0,False,58,management,married,tertiary,False,2143,True,False
1,False,44,technician,single,secondary,False,29,True,False
2,False,33,entrepreneur,married,secondary,False,2,True,True
3,False,47,blue-collar,married,NaN,False,1506,True,False
4,False,33,NaN,single,NaN,False,1,False,False


### 演習問題

1. 「住宅ローン」の有無が、今回販促結果に影響するかどうかを、groupbyメソッドで調べて下さい
2. 「住宅ローン」「学歴」の2軸で頻度分析をして下さい
3. 「住宅ローン」「学歴」の2軸で「今回販促結果」のクロス集計をして下さい

In [9]:
# 1. 住宅ローンの有無が、今回販促結果に影響するか
# 参照 3.4.5項　コード3-4-11
df_gr = df2.groupby('住宅ローン').mean()

# 結果確認
display(df_gr)

,今回販促結果,年齢,債務不履行,平均残高,個人ローン
住宅ローン,,,,,
False,0.1670,43.1399,0.0189,1596.5013,0.1433
True,0.0770,39.1753,0.0173,1175.1031,0.1738


In [10]:
# 2. 「住宅ローン」「学歴」の2軸で頻度分析
# 参照 3.4．6項 コード 3-4-13
df_crosstab = pd.crosstab(
    index=df2['住宅ローン'], 
    columns=df2['学歴'],
    margins=True)

# 結果確認
display(df_crosstab)

学歴,primary,secondary,tertiary,All
住宅ローン,,,,
False,2957,9164,6923,19044
True,3894,14038,6378,24310
All,6851,23202,13301,43354


In [11]:
# 3. 「住宅ローン」「学歴」の2軸で「今回販促結果」のクロス集計
# 参照　3.4.7項　コード3-4-15
df_pivot = df2.pivot_table(
    index='住宅ローン', 
    columns='学歴',
    values='今回販促結果', 
    aggfunc='mean')

# 結果確認
display(df_pivot)

学歴,primary,secondary,tertiary
住宅ローン,,,
False,0.1238,0.1542,0.1996
True,0.0578,0.0739,0.0963
